In [3]:
import os
import re
from glob import glob

output_root = "./recordings/"
dz_paths = glob(output_root + '*/*/capspeaker.txt')

with open(dz_paths[0], "r") as txt:
    dialogues = txt.read()
    dialogues = dialogues.splitlines()
dialogues[:10]

['[00:00:000.00 --> 00:00:000.78] [SPEAKER_02]  How are you doing today?',
 '[00:00:001.62 --> 00:00:002.42] [SPEAKER_02]  Doing great.',
 '[00:00:002.42 --> 00:00:003.20] [SPEAKER_02]  Doing great.',
 '[00:00:003.34 --> 00:00:003.68] [SPEAKER_02]  Is your name Brittany?',
 '[00:00:004.06 --> 00:00:004.44] [SPEAKER_02]  Is your name Brittany?',
 '[00:00:004.82 --> 00:00:005.58] [SPEAKER_02]  Okay, perfect.',
 "[00:00:005.62 --> 00:00:005.94] [SPEAKER_02]  I'm Jessica.",
 '[00:00:005.94 --> 00:00:006.38] [SPEAKER_02]  Okay, perfect.',
 "[00:00:006.52 --> 00:00:006.90] [SPEAKER_02]  I'm Jessica.",
 "[00:00:007.14 --> 00:00:009.68] [SPEAKER_02]  I'll help you start the exam before you see the doctor today."]

In [4]:
from diarization_utils import CaptionCleaner, millisec, timeStr

cleaner = CaptionCleaner()
cleaned = cleaner.condense(dialogues)
cleaned[:10]

Dialogue contains : 4 speakers
{'SPEAKER_00', 'SPEAKER_02', 'SPEAKER_01', 'SPEAKER_03'}
removed 281 lines.


["[SPEAKER_02]  How are you doing today?  Doing great.  Is your name Brittany?  Okay, perfect.  I'm Jessica.  Okay, perfect.  I'm Jessica.  I'll help you start the exam before you see the doctor today.  Okay.  How are you doing today?  Doing great.  Is your name Brittany?  Okay, perfect.  I'm Jessica.  Okay, perfect.  I'm Jessica.  I'll help you start the exam before you see the doctor today.  Okay.  How are you doing today?  Doing great.  Is your name Brittany?  Okay, perfect.  I'm Jessica.  Okay, perfect.  I'm Jessica.  I'll help you start the exam before you see the doctor today.  Okay.  How are you doing today?  Doing great.  Is your name Brittany?  Okay, perfect.  I'm Jessica.  Okay, perfect.  I'm Jessica.  I'll help you start the exam before you see the doctor today.  Okay.  How are you doing today?  Doing great.  Is your name Brittany?  Okay, perfect.  I'm Jessica.  Okay, perfect.  I'm Jessica.  I'll help you start the exam before you see the doctor today.  Okay.",
 '[SPEAKER_03

# Fixing diarization misalignment

In [52]:
dz_paths = glob(output_root + '*/*/diarization.txt')
print(f'found {len(dz_paths)} diarizations')

found 334 diarizations


In [53]:
dzs = open(dz_paths[0]).read().splitlines()
print(dz_paths[0])
dzs[:10]

./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/diarization.txt


['[ 00:00:01.960 -->  00:00:02.996] A SPEAKER_02',
 '[ 00:00:04.337 -->  00:00:05.441] B SPEAKER_02',
 '[ 00:00:05.899 -->  00:00:06.799] C SPEAKER_02',
 '[ 00:00:07.852 -->  00:00:11.977] D SPEAKER_02',
 '[ 00:00:12.096 -->  00:00:12.911] E SPEAKER_03',
 '[ 00:00:12.911 -->  00:00:14.134] F SPEAKER_02',
 '[ 00:00:15.984 -->  00:00:17.546] G SPEAKER_03',
 '[ 00:00:20.993 -->  00:00:22.843] H SPEAKER_03',
 '[ 00:00:23.047 -->  00:00:23.488] I SPEAKER_03',
 '[ 00:00:28.446 -->  00:00:29.363] J SPEAKER_03']

In [58]:
def deduplicate(groups):
    old = []
    deduped = []
    for g in groups:
        if old != g:
            old = g
            deduped.append(g)
    return deduped

groups = []
g = []
lastend = 0
for d in dzs:
    # same speaker
    if g and (g[0].split()[-1] != d.split()[-1]): 
        groups.append(g)
        g = []
    
    g.append(d)
    
    end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=d)[1]
    end = millisec(end)
    if (lastend > end):     #segment engulfed by a previous segment
        groups.append(g)
        g = []
    else:
        lastend = end
    if g:
        groups.append(g)
groups = deduplicate(groups)
print(*groups, sep='\n')

['[ 00:00:01.960 -->  00:00:02.996] A SPEAKER_02', '[ 00:00:04.337 -->  00:00:05.441] B SPEAKER_02', '[ 00:00:05.899 -->  00:00:06.799] C SPEAKER_02', '[ 00:00:07.852 -->  00:00:11.977] D SPEAKER_02']
['[ 00:00:12.096 -->  00:00:12.911] E SPEAKER_03']
['[ 00:00:12.911 -->  00:00:14.134] F SPEAKER_02']
['[ 00:00:15.984 -->  00:00:17.546] G SPEAKER_03', '[ 00:00:20.993 -->  00:00:22.843] H SPEAKER_03', '[ 00:00:23.047 -->  00:00:23.488] I SPEAKER_03', '[ 00:00:28.446 -->  00:00:29.363] J SPEAKER_03', '[ 00:00:29.533 -->  00:00:29.702] K SPEAKER_03']
['[ 00:00:38.208 -->  00:00:46.375] L SPEAKER_02']
['[ 00:00:47.003 -->  00:00:50.110] M SPEAKER_03', '[ 00:00:50.466 -->  00:00:50.534] N SPEAKER_03']
['[ 00:00:50.534 -->  00:00:53.913] O SPEAKER_02']
['[ 00:00:50.568 -->  00:00:50.806] P SPEAKER_03']
['[ 00:00:53.166 -->  00:00:58.531] Q SPEAKER_03']
['[ 00:00:58.123 -->  00:00:58.378] R SPEAKER_02']
['[ 00:00:58.870 -->  00:00:59.804] S SPEAKER_03']
['[ 00:00:59.821 -->  00:01:01.842] T S

In [59]:
from pydub import AudioSegment

fpath = '/'.join(dz_paths[0].split('/')[:-1])
print(f'reading wav file: {fpath + "/input_prep.wav"}')
audio = AudioSegment.from_wav(fpath + "/input_prep.wav")
gidx = -1
for g in groups:
    start = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
    end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[-1])[1]
    start = millisec(start)
    end = millisec(end)
    gidx += 1
    audio[start:end].export(fpath + '/' + str(gidx) + '.wav', format='wav')

reading wav file: ./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/input_prep.wav


In [60]:
import whisper
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = whisper.load_model('large', device = device)

In [61]:
import json

for i in range(len(groups)):
    audiof = fpath + '/' + str(i) + '.wav'
    result = model.transcribe(audio=audiof, language='en', word_timestamps=True)#, initial_prompt=result.get('text', ""))
    with open(fpath + '/' + str(i)+'.json', "w") as outfile:
        json.dump(result, outfile, indent=4)

In [62]:
from diarization_utils import *

In [63]:
spacermilli = 2000
speakers = {'SPEAKER_00':('SPEAKER_00', '#e1ffc7', 'darkgreen'), 'SPEAKER_01':('SPEAKER_01', 'white', 'darkorange') }
def_boxclr = 'white'
def_spkrclr = 'orange'
html = list(preS)
txt = list("")
gidx = -1
for g in groups:  
    shift = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
    shift = millisec(shift) - spacermilli #the start time in the original video
    shift=max(shift, 0)
    
    gidx += 1
    
    captions = json.load(open(fpath + '/' + str(gidx) + '.json'))['segments']

    if captions:
        speaker = g[0].split()[-1]
        boxclr = def_boxclr
        spkrclr = def_spkrclr
        if speaker in speakers:
            speaker, boxclr, spkrclr = speakers[speaker] 
    
        html.append(f'<div class="e" style="background-color: {boxclr}">\n');
        html.append('<p  style="margin:0;padding: 5px 10px 10px 10px;word-wrap:normal;white-space:normal;">\n')
        html.append(f'<span style="color:{spkrclr};font-weight: bold;">{speaker}</span><br>\n\t\t\t\t')
        
        for c in captions:
            start = shift + c['start'] * 1000.0 
            start = start / 1000.0   #time resolution ot youtube is Second.            
            end = (shift + c['end'] * 1000.0) / 1000.0      
            txt.append(f'[{timeStr(start)} --> {timeStr(end)}] [{speaker}] {c["text"]}\n')

            for i, w in enumerate(c['words']):
                if w == "":
                    continue
                start = (shift + w['start']*1000.0) / 1000.0        
                #end = (shift + w['end']) / 1000.0   #time resolution ot youtube is Second.  
                html.append(f'<a href="#{timeStr(start)}" id="{"{:.1f}".format(round(start*5)/5)}" class="lt" onclick="jumptoTime({int(start)}, this.id)">{w["word"]}</a><!--\n\t\t\t\t-->')
            #html.append('\n')      
            html.append('</p>\n')
            html.append(f'</div>\n')

        html.append(postS)

        capclean = CaptionCleaner()
        with open(f"{fpath}/capspeaker.txt", "w", encoding='utf-8') as file:
            s = "".join(txt)
            file.write(s)
            print(f'captions saved to /{fpath}/capspeaker.txt:')
        with open(f"{fpath}/capspeaker.html", "w", encoding='utf-8') as file:    #TODO: proper html embed tag when video/audio from file
            s = "".join(html)
            file.write(s)
            print(f'html saved to {fpath}/capspeaker.html:')

captions saved to /./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.txt:
html saved to ./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.html:
captions saved to /./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.txt:
html saved to ./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.html:
captions saved to /./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.txt:
html saved to ./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.html:
captions saved to /./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.txt:
html saved to ./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.html:
captions saved to /./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.txt:
html saved to ./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.html:
captions saved to /./recordings/RT-JJara

In [1]:
from glob import glob
from diarization_utils import CaptionCleaner

output_root = "/home/digitalopt/proj/diarization/recordings/"
cap_paths = glob(output_root + '*/*/capspeaker.txt')

cap_paths.sort()
cap_paths[1]


/home/digitalopt/miniconda3/envs/diarization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/digitalopt/proj/diarization/recordings/RT-JJara/1040390_6190_VC_1_1_29_11_2023_14_17_42/capspeaker.txt'

In [2]:
cleaner.NER == True

NameError: name 'cleaner' is not defined

In [3]:
cleaner = CaptionCleaner()    
captions = cleaner.read_captions(cap_paths[1])
cleaned = cleaner.clean(captions)
cleaned[:25]
# outfile = '/'.join(cap_paths[0].split('/')[:-1]) + '/cleaner_caps.txt'
# with open(outfile, 'w') as file:
#     output = '\n'.join(cleaned)
#     file.write(output)

spacy model en_core_web_trf loaded
removed 75 lines.
Dialogue contains : 5 speakers.
speaker: SPEAKER_02, words spoken: 486
speaker: SPEAKER_01, words spoken: 109
speaker: SPEAKER_00, words spoken: 98
speaker: SPEAKER_04, words spoken: 46
speaker: SPEAKER_03, words spoken: 320
More than three speakers in captions! Only 3 speakers allowed.
sorted speakers by word count: 
[('SPEAKER_04', 46), ('SPEAKER_00', 98), ('SPEAKER_01', 109), ('SPEAKER_03', 320), ('SPEAKER_02', 486)]
/home/digitalopt/proj/diarization/recordings/RT-JJara/1040390_6190_VC_1_1_29_11_2023_14_17_42/capspeaker.txt is invalid! Only 3 speakers allowed. Removing speakers by lowest word count.
truncated speaker list: ['SPEAKER_01', 'SPEAKER_03', 'SPEAKER_02']
speaker mapping <spaker in data> : <new spaker label>
{'SPEAKER_01': 'LocalTech', 'SPEAKER_03': 'Patient__', 'SPEAKER_02': 'Assistant'}
removing names...


["[Assistant]  Hi, how's everyone doing today?",
 "[LocalTech]  How's everyone doing today?  Good. How are you?",
 '[Assistant]  Doing great.',
 '[LocalTech]  Doing great.',
 "[Assistant]  Is this [NAME]? Yes, ma'am. Nice to meet you. My name is [NAME]. I'll help you start your exam before you see the doctor today.",
 "[LocalTech]  Nice to meet you.  My name is [NAME].  I'll help you start your exam before you see the doctor.",
 '[LocalTech]',
 "[SPEAKER_02]  And we'll get you set up in the chair first.",
 '[SPEAKER_00]  set up in the chair first.',
 "[SPEAKER_01]  Is that comfortable?  You think?  Yeah.  Yeah, I'm gonna fix your feet.  Okay, thanks.  You're welcome.  You're welcome.  Can I go in on his left eye a tad bit?",
 '[SPEAKER_02]  Yeah, sure. Is this better?  Yes.',
 "[SPEAKER_01]  You're going to.",
 '[SPEAKER_04]  Eyelashes are important.',
 '[SPEAKER_01]  I know they point down.  Does that feel comfortable?  Yeah.  He looks good.  Okay, thank you.  Thank you.',
 "[SPEAKER_

In [3]:
cleaned[:10]

["[Assistant]  How are you doing today?  Doing great.  Is your name Brittany?  Okay, perfect.  I'm Jessica.  Okay, perfect.  I'm Jessica.  I'll help you start the exam before you see the doctor today.  Okay.",
 '[Patient__]  It just set up.',
 "[Assistant]  We'll get you set up in the chair.",
 "[Patient__]  And then are you sitting comfortable?  Yeah.  Bear with me.  I'm still training on this part.  You're good.  Is that okay?  Yeah.",
 "[Assistant]  Thank you. Okay, so you should see three lines of letters with no squinting. What's the smallest line you're able to read?",
 '[Patient__]  O-Z-D-V-K.  Good.',
 '[Assistant]  Good. Anything here on this line or is this a little too blurry?',
 "[Patient__]  guy.  too blurry.  W-Z-O-D-C?  Sorry, that's an F.",
 '[Assistant]  guys.',
 '[Patient__]  Okay, perfect.']

In [68]:
thing1 = cleaner.remove_time(captions)
thing1[:5]

['[SPEAKER_02]  How are you doing today?',
 '[SPEAKER_02]  Doing great.',
 '[SPEAKER_02]  Doing great.',
 '[SPEAKER_02]  Is your name Brittany?',
 '[SPEAKER_02]  Is your name Brittany?']

In [69]:
thing2 = cleaner.deduplicate(thing1)
thing2[:5]

['[SPEAKER_02]  How are you doing today?',
 '[SPEAKER_02]  Doing great.',
 '[SPEAKER_02]  Is your name Brittany?',
 '[SPEAKER_02]  Okay, perfect.',
 "[SPEAKER_02]  I'm Jessica."]

In [70]:
thing2[:20]

['[SPEAKER_02]  How are you doing today?',
 '[SPEAKER_02]  Doing great.',
 '[SPEAKER_02]  Is your name Brittany?',
 '[SPEAKER_02]  Okay, perfect.',
 "[SPEAKER_02]  I'm Jessica.",
 '[SPEAKER_02]  Okay, perfect.',
 "[SPEAKER_02]  I'm Jessica.",
 "[SPEAKER_02]  I'll help you start the exam before you see the doctor today.",
 '[SPEAKER_02]  Okay.',
 '[SPEAKER_03]  It just set up.',
 "[SPEAKER_02]  We'll get you set up in the chair.",
 '[SPEAKER_03]  And then are you sitting comfortable?',
 '[SPEAKER_03]  Yeah.',
 '[SPEAKER_03]  Bear with me.',
 "[SPEAKER_03]  I'm still training on this part.",
 "[SPEAKER_03]  You're good.",
 '[SPEAKER_03]  Is that okay?',
 '[SPEAKER_03]  Yeah.',
 "[SPEAKER_02]  Thank you. Okay, so you should see three lines of letters with no squinting. What's the smallest line you're able to read?",
 '[SPEAKER_03]  O-Z-D-V-K.']

In [71]:
def condense(txt):
    condensed = []
    old_speaker = ''
    line = ''
    for idx in range(len(txt)):
        speaker = txt[idx][1:11]
        if old_speaker == speaker:
            line += txt[idx][12:]
        else:
            if line != '':
                condensed.append(line)
            old_speaker = speaker
            line = f'[{speaker}]'
            line += txt[idx][12:]
    return condensed

thing3 = condense(thing2)
thing3[:10]

["[SPEAKER_02]  How are you doing today?  Doing great.  Is your name Brittany?  Okay, perfect.  I'm Jessica.  Okay, perfect.  I'm Jessica.  I'll help you start the exam before you see the doctor today.  Okay.",
 '[SPEAKER_03]  It just set up.',
 "[SPEAKER_02]  We'll get you set up in the chair.",
 "[SPEAKER_03]  And then are you sitting comfortable?  Yeah.  Bear with me.  I'm still training on this part.  You're good.  Is that okay?  Yeah.",
 "[SPEAKER_02]  Thank you. Okay, so you should see three lines of letters with no squinting. What's the smallest line you're able to read?",
 '[SPEAKER_03]  O-Z-D-V-K.  Good.',
 '[SPEAKER_02]  Good. Anything here on this line or is this a little too blurry?',
 "[SPEAKER_03]  guy.  too blurry.  W-Z-O-D-C?  Sorry, that's an F.",
 '[SPEAKER_02]  guys.',
 '[SPEAKER_03]  Okay, perfect.']

# After manually editing the data

In [1]:
from glob import glob
from diarization_utils import CaptionCleaner

/home/digitalopt/miniconda3/envs/diarization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = '/data/datasets/diarization_recordings/processed_recordings/'
jjara = data_dir + 'RT-JJara/'
stephens = data_dir + 'rt-lstephens/'
brokus = data_dir + 'rt-sbrokus/'

jjara = glob(jjara + '*/reviewed_caps.txt')
stephens = glob(stephens + '*/reviewed_caps.txt')
brokus = glob(brokus + '*/reviewed_caps.txt')
for i in [jjara, stephens, brokus]:
    print (len(i))

5
5
5


In [9]:
import shutil
new_root = '/data/datasets/Exam_v3/'
cnt = 30
for fgrp in [jjara, stephens, brokus]:
    for f in fgrp:
        if cnt % 3 != 0:
            fpath = new_root + 'val/' + str(cnt).zfill(6) + '.txt'
            shutil.copy(f,fpath)
        else: 
            fpath = new_root + 'test/' + str(cnt).zfill(6) + '.txt'
            shutil.copy(f,fpath)
        cnt += 1

In [4]:
cleaner = CaptionCleaner()
cleaner.load_NER_model()
output_root = "/home/digitalopt/proj/diarization/recordings/"
cap_paths = glob(output_root + '*/*/capspeaker.txt')
for p in cap_paths:
    print(f'reading {p}')
    captions = cleaner.read_captions(p)
    cleaned = cleaner.clean(captions)
    outfile = '/'.join(p.split('/')[:-1]) + '/cleaner_caps.txt'
    cleaner.write_captions(cleaned, outfile)

spacy model en_core_web_trf loaded
reading /home/digitalopt/proj/diarization/recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/capspeaker.txt
removed 74 lines.
speakers keys: dict_keys([])
Dialogue contains : 3 speakers.
speaker: SPEAKER_02, words spoken: 539
speaker: SPEAKER_03, words spoken: 210
speaker: SPEAKER_01, words spoken: 33
sorted speakers by word count: 
[('SPEAKER_01', 33), ('SPEAKER_03', 210), ('SPEAKER_02', 539)]
removing names...
reading /home/digitalopt/proj/diarization/recordings/RT-JJara/595841_4602_VC_1_1_01_12_2023_14_34_27/capspeaker.txt
removed 32 lines.
speakers keys: dict_keys([])
Dialogue contains : 4 speakers.
speaker: SPEAKER_03, words spoken: 405
speaker: SPEAKER_01, words spoken: 46
speaker: SPEAKER_02, words spoken: 61
speaker: SPEAKER_00, words spoken: 1
More than three speakers in captions! Only 3 speakers allowed.
sorted speakers by word count: 
[('SPEAKER_00', 1), ('SPEAKER_01', 46), ('SPEAKER_02', 61), ('SPEAKER_03', 405)]
/home/digitalopt

KeyError: 'SPEAKER_00'

remapping speaker names:

- SPEAKER_00 : local tech
- SPEAKER_01 : patient
- SPEAKER_02 : assistant

if speaker had greatest words spoken, they are the assistant.
if speaker had 2nd most words, they are the patient
if speaker spoke the least, they are the local tech.

first, identify the speaker names and words spoken in the caption set.
second, map the speaker names to new identifiers
second, for each caption, identify the speaker and replace the speaker name according to the new mapping.

In [4]:
captions2 = captions.copy()
print(captions2[:6])
thing = cleaner.remap_speaker_names(captions2)
thing[:10]

['[SPEAKER_02]  How are you doing today?', '[SPEAKER_03]  Doing great.', '[SPEAKER_02]  Is your name Brittany?', '[SPEAKER_03]  Yes.', "[SPEAKER_02]  Okay, perfect.  I'm Jessica. I'll help you start the exam before you see the doctor today. We'll get you set up in the chair.", '[SPEAKER_01]  And then are you sitting comfortable?']
sorted speakers by word count: 
[('SPEAKER_01', 23), ('SPEAKER_03', 121), ('SPEAKER_02', 514)]


['[SPEAKER_02]  How are you doing today?',
 '[SPEAKER_01]  Doing great.',
 '[SPEAKER_02]  Is your name Brittany?',
 '[SPEAKER_01]  Yes.',
 "[SPEAKER_02]  Okay, perfect.  I'm Jessica. I'll help you start the exam before you see the doctor today. We'll get you set up in the chair.",
 '[SPEAKER_00]  And then are you sitting comfortable?',
 '[SPEAKER_01]  Yeah.',
 "[SPEAKER_00]  Bear with me.  I'm still training on this part.  You're good.  Is that okay?",
 '[SPEAKER_01]  Yeah.',
 "[SPEAKER_02]  Thank you. Okay, so you should see three lines of letters with no squinting. What's the smallest line you're able to read?"]

In [5]:
def new_filepath(fpath):
    fname = fpath.split('/')[:-1]
    fname.append('speaker_captions.txt')
    fname = '/'.join(fname)
    return fname
jjara = data_dir + 'RT-JJara/'
jjara = glob(jjara + '*/clean_captions.txt')[:10]

path_groups = [jjara, stephens, brokus]
for group in path_groups:
    for path in group:
        print(f'Processing {path}')
        newfile = new_filepath(path)
        captions = cleaner.read_captions(path)
        speakers = cleaner.speaker_cnt(captions)
        captions = cleaner.remap_speaker_names(captions)
        cleaner.write_captions(captions, newfile)

Processing ./recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/clean_captions.txt
speakers keys: dict_keys([])
Dialogue contains : 3 speakers.
speaker: SPEAKER_02, words spoken: 514
speaker: SPEAKER_03, words spoken: 121
speaker: SPEAKER_01, words spoken: 23
sorted speakers by word count: 
[('SPEAKER_01', 23), ('SPEAKER_03', 121), ('SPEAKER_02', 514)]
Processing ./recordings/RT-JJara/595841_4602_VC_1_1_01_12_2023_14_34_27/clean_captions.txt
speakers keys: dict_keys([])
Dialogue contains : 3 speakers.
speaker: SPEAKER_03, words spoken: 362
speaker: SPEAKER_02, words spoken: 51
speaker: SPEAKER_01, words spoken: 29
sorted speakers by word count: 
[('SPEAKER_01', 29), ('SPEAKER_02', 51), ('SPEAKER_03', 362)]
Processing ./recordings/RT-JJara/1932708_11628_VC_1_1_04_12_2023_16_02_48/clean_captions.txt
speakers keys: dict_keys([])
Dialogue contains : 3 speakers.
speaker: SPEAKER_02, words spoken: 592
speaker: SPEAKER_01, words spoken: 93
speaker: SPEAKER_00, words spoken: 1
sorted

In [6]:
fname = './recordings/rt-sbrokus/566198_1378_VC_1_1_28_11_2023_16_38_04/clean_captions.txt'
captions = cleaner.read_captions(fname)
speakers = cleaner.speaker_cnt(captions)
# captions = cleaner.remap_speaker_names(captions)
# cleaner.write_captions(captions, newfile)

speakers keys: dict_keys([])
Dialogue contains : 3 speakers.
speaker: SPEAKER_01, words spoken: 564
speaker: SPEAKER_02, words spoken: 68
speaker: SPEAKER_03, words spoken: 142


# Scrub Names

In [7]:
import en_core_web_trf
import spacy_transformers
import spacy

/home/digitalopt/miniconda3/envs/diarization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
nlp = spacy.load("en_core_web_trf")

In [9]:
captions = cleaner.read_captions(jjara[0])
captions[:10]

['[SPEAKER_02]  How are you doing today?',
 '[SPEAKER_03]  Doing great.',
 '[SPEAKER_02]  Is your name Brittany?',
 '[SPEAKER_03]  Yes.',
 "[SPEAKER_02]  Okay, perfect.  I'm Jessica. I'll help you start the exam before you see the doctor today. We'll get you set up in the chair.",
 '[SPEAKER_01]  And then are you sitting comfortable?',
 '[SPEAKER_03]  Yeah.',
 "[SPEAKER_01]  Bear with me.  I'm still training on this part.  You're good.  Is that okay?",
 '[SPEAKER_03]  Yeah.',
 "[SPEAKER_02]  Thank you. Okay, so you should see three lines of letters with no squinting. What's the smallest line you're able to read?"]

In [1]:
from glob import glob
from diarization_utils import CaptionCleaner

output_root = "/home/digitalopt/proj/diarization/recordings/"
cap_paths = glob(output_root + '*/*/clean_captions.txt')
print(f'found {len(cap_paths)} caption texts')
cleaner = CaptionCleaner()
cleaner.load_NER_model()
for p in cap_paths:
    print(f'reading {p}')
    captions = cleaner.read_captions(p)
    cleaned = cleaner.clean(captions, full_clean=False)
    if cleaned:
        outfile = '/'.join(p.split('/')[:-1]) + '/clean_captions1.txt'
        cleaner.write_captions(cleaned, outfile)

/home/digitalopt/miniconda3/envs/diarization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


found 40 caption texts
spacy model en_core_web_trf loaded
reading /home/digitalopt/proj/diarization/recordings/RT-JJara/2031456_11398_VC_1_1_01_12_2023_15_03_31/clean_captions.txt
Dialogue contains : 3 speakers.
speaker: SPEAKER_02, words spoken: 514
speaker: SPEAKER_03, words spoken: 121
speaker: SPEAKER_01, words spoken: 23
sorted speakers by word count: 
[('SPEAKER_01', 23), ('SPEAKER_03', 121), ('SPEAKER_02', 514)]
removing names...
reading /home/digitalopt/proj/diarization/recordings/RT-JJara/595841_4602_VC_1_1_01_12_2023_14_34_27/clean_captions.txt
Dialogue contains : 3 speakers.
speaker: SPEAKER_03, words spoken: 362
speaker: SPEAKER_02, words spoken: 51
speaker: SPEAKER_01, words spoken: 29
sorted speakers by word count: 
[('SPEAKER_01', 29), ('SPEAKER_02', 51), ('SPEAKER_03', 362)]
removing names...
reading /home/digitalopt/proj/diarization/recordings/RT-JJara/1932708_11628_VC_1_1_04_12_2023_16_02_48/clean_captions.txt
Dialogue contains : 3 speakers.
speaker: SPEAKER_02, words

## Logging vision exam stages

In [5]:
from glob import glob

data_dir = '/home/digitalopt/proj/datasets/Exam_v2/'
fpaths = glob(data_dir + '*/*.txt')
fpaths.sort()
fpaths[:5]

['/home/digitalopt/proj/datasets/Exam_v2/test/000024.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/test/000025.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/test/000026.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/test/000027.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/test/000028.txt']

In [12]:
fpaths[6:27]

['/home/digitalopt/proj/datasets/Exam_v2/train/000000.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000001.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000002.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000003.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000004.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000005.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000006.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000007.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000008.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000009.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000010.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000011.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000012.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000013.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000014.txt',
 '/home/digitalopt/proj/datasets/Exam_v2/train/000015.txt',
 '/home/digitalopt/proj/datasets/Exam_v2

In [6]:
from collections import defaultdict

template = dict(greeting_setup=(),
                va_unaided=(),
                subjective_refraction=(),
                close_vision=()  
)

exam_by_stage = defaultdict(list)

for p in fpaths:
    exam_by_stage[p] = template

print(fpaths[6], exam_by_stage[fpaths[6]])


/home/digitalopt/proj/datasets/Exam_v2/train/000000.txt {'greeting_setup': (), 'va_unaided': (), 'subjective_refraction': (), 'close_vision': ()}


In [7]:

exam_by_stage[fpaths[6]]['greeting_setup'] = (0,17)
exam_by_stage[fpaths[6]]['va_unaided'] = (17,27)
exam_by_stage[fpaths[6]]['subjective_refraction'] = (27,61)
exam_by_stage[fpaths[6]]['close_vision'] = (61,67)

exam_by_stage[fpaths[7]]['greeting_setup'] = (0,9)
exam_by_stage[fpaths[7]]['va_unaided'] = (9,23)
exam_by_stage[fpaths[7]]['subjective_refraction'] = (23,69)
exam_by_stage[fpaths[7]]['close_vision'] = (69,73)

exam_by_stage[fpaths[8]]['greeting_setup'] = (0,16)
exam_by_stage[fpaths[8]]['va_unaided'] = (16,26)
exam_by_stage[fpaths[8]]['subjective_refraction'] = (26,59)
exam_by_stage[fpaths[8]]['close_vision'] = (59,66)

exam_by_stage[fpaths[9]]['greeting_setup'] = (0,9)
exam_by_stage[fpaths[9]]['va_unaided'] = (9,19)
exam_by_stage[fpaths[9]]['subjective_refraction'] = (19,68)
exam_by_stage[fpaths[9]]['close_vision'] = (68,72)

exam_by_stage[fpaths[10]]['greeting_setup'] = (0,11)
exam_by_stage[fpaths[10]]['va_unaided'] = (11,21)
exam_by_stage[fpaths[10]]['subjective_refraction'] = (21,65)
exam_by_stage[fpaths[10]]['close_vision'] = (65,75)

exam_by_stage[fpaths[11]]['greeting_setup'] = (0,9)
exam_by_stage[fpaths[11]]['va_unaided'] = (9,21)
exam_by_stage[fpaths[11]]['subjective_refraction'] = (21,77)
exam_by_stage[fpaths[11]]['close_vision'] = (77,83)

exam_by_stage[fpaths[12]]['greeting_setup'] = (0,11)
exam_by_stage[fpaths[12]]['va_unaided'] = (11,21)
exam_by_stage[fpaths[12]]['subjective_refraction'] = (21,37)
exam_by_stage[fpaths[12]]['close_vision'] = (37,43)

exam_by_stage[fpaths[13]]['greeting_setup'] = (0,11)
exam_by_stage[fpaths[13]]['va_unaided'] = (11,32)
exam_by_stage[fpaths[13]]['subjective_refraction'] = (32,60)
exam_by_stage[fpaths[13]]['close_vision'] = (60,62)

exam_by_stage[fpaths[14]]['greeting_setup'] = (0,11)
exam_by_stage[fpaths[14]]['va_unaided'] = (11,27)
exam_by_stage[fpaths[14]]['subjective_refraction'] = (27,61)
exam_by_stage[fpaths[14]]['close_vision'] = (61,68)

exam_by_stage[fpaths[15]]['greeting_setup'] = (0,12)
exam_by_stage[fpaths[15]]['va_unaided'] = (12,34)
exam_by_stage[fpaths[15]]['subjective_refraction'] = (34,56)
exam_by_stage[fpaths[15]]['close_vision'] = (56,67)

exam_by_stage[fpaths[16]]['greeting_setup'] = (0,11)
exam_by_stage[fpaths[16]]['va_unaided'] = (11,25)
exam_by_stage[fpaths[16]]['subjective_refraction'] = (25,57)
exam_by_stage[fpaths[16]]['close_vision'] = (57,63)

exam_by_stage[fpaths[17]]['greeting_setup'] = (0,7)
exam_by_stage[fpaths[17]]['va_unaided'] = (7,17)
exam_by_stage[fpaths[17]]['subjective_refraction'] = (17,55)
exam_by_stage[fpaths[17]]['close_vision'] = (55,58)

exam_by_stage[fpaths[18]]['greeting_setup'] = (0,5)
exam_by_stage[fpaths[18]]['va_unaided'] = (5,19)
exam_by_stage[fpaths[18]]['subjective_refraction'] = (19,49)
exam_by_stage[fpaths[18]]['close_vision'] = (49,55)

exam_by_stage[fpaths[19]]['greeting_setup'] = (0,3)
exam_by_stage[fpaths[19]]['va_unaided'] = (3,15)
exam_by_stage[fpaths[19]]['subjective_refraction'] = (15,49)
exam_by_stage[fpaths[19]]['close_vision'] = (49,56)

exam_by_stage[fpaths[20]]['greeting_setup'] = (0,7)
exam_by_stage[fpaths[20]]['va_unaided'] = (7,33)
exam_by_stage[fpaths[20]]['subjective_refraction'] = (33,63)
exam_by_stage[fpaths[20]]['close_vision'] = (63,69)

exam_by_stage[fpaths[21]]['greeting_setup'] = (0,11)
exam_by_stage[fpaths[21]]['va_unaided'] = (11,27)
exam_by_stage[fpaths[21]]['subjective_refraction'] = (27,75)
exam_by_stage[fpaths[21]]['close_vision'] = (75,81)

exam_by_stage[fpaths[22]]['greeting_setup'] = (0,11)
exam_by_stage[fpaths[22]]['va_unaided'] = (11,29)
exam_by_stage[fpaths[22]]['subjective_refraction'] = (29,71)
exam_by_stage[fpaths[22]]['close_vision'] = (71,83)

exam_by_stage[fpaths[23]]['greeting_setup'] = (0,16)
exam_by_stage[fpaths[23]]['va_unaided'] = (16,36)
exam_by_stage[fpaths[23]]['subjective_refraction'] = (36,72)
exam_by_stage[fpaths[23]]['close_vision'] = (72,78)

exam_by_stage[fpaths[24]]['greeting_setup'] = (0,15)
exam_by_stage[fpaths[24]]['va_unaided'] = (15,33)
exam_by_stage[fpaths[24]]['subjective_refraction'] = (33,79)
exam_by_stage[fpaths[24]]['close_vision'] = (79,85)

exam_by_stage[fpaths[25]]['greeting_setup'] = (0,13)
exam_by_stage[fpaths[25]]['va_unaided'] = (13,41)
exam_by_stage[fpaths[25]]['subjective_refraction'] = (41,73)
exam_by_stage[fpaths[25]]['close_vision'] = (73,81)

exam_by_stage[fpaths[26]]['greeting_setup'] = (0,9)
exam_by_stage[fpaths[26]]['va_unaided'] = (9,23)
exam_by_stage[fpaths[26]]['subjective_refraction'] = (23,55)
exam_by_stage[fpaths[26]]['close_vision'] = (55,59)


In [8]:
greet_lens = []
va_lens = []
sr_lens = []
cv_lens = []
results = dict(greeting_setup=[],
                va_unaided=[],
                subjective_refraction=[],
                close_vision=[],
                cap_length=[]
)

for p in fpaths[6:27]:
    for k in exam_by_stage[p].keys():
        length = exam_by_stage[p][k][1] - exam_by_stage[p][k][0]
        results[k].append(length)
        # the endpoint of close_vision is the last index
        if k == "close_vision":
            # add 1 to last index to get length of dialogue
            results["cap_length"].append(exam_by_stage[p][k][1] + 1)

In [9]:
for k in results.keys():
    avg = sum(results[k]) / len(results[k])
    print(k,avg)
    # 3364705

greeting_setup 9.0
va_unaided 14.0
subjective_refraction 32.0
close_vision 4.0
cap_length 60.0


In [14]:
avg_length = sum(results["cap_length"]) / len(results["cap_length"])

print('Average percent of each dialogue that each section represents:\n')
for k in results.keys():
    avg = sum(results[k]) / len(results[k])
    print(k,round((avg/avg_length)*100, 2))

Average percent of each dialogue that each section represents:

greeting_setup 15.0
va_unaided 23.33
subjective_refraction 53.33
close_vision 6.67
cap_length 100.0


In [16]:
15+24+54+7

100

In [21]:
3/27

0.1111111111111111

In [19]:
4/27

0.36363636363636365